In [1]:
import sys

from opts import get_opts
from utils import get_data_base_path, make_main_figure, notebook_config

sys.path.insert(1, "/Users/user/Documents/Git_repos/climate-epidemics")

import climepi  # noqa
from climepi import climdata, epimod

# Configure notebook
notebook_config()

# Get options
options = get_opts()
year_range = options["year_range"]
location = options["location_default"]
epi_model_name = options["epi_model_name_default"]
polyfit_degree = options["polyfit_degree"]
plot_opts_temp = options["plot_opts_temp"]
plot_opts_epi = options["plot_opts_epi"]
figure_dir = options["figure_dir"]

In [2]:
# Get climate data
ds_clim = (
    climdata.get_example_dataset(
        "isimip_cities", get_data_base_path() / "isimip_cities"
    )
    .sel(location=location)
    .sel(time=slice(str(year_range[0]), str(year_range[1])))
)
# Yearly average temperature data
ds_temp_yearly = ds_clim.climepi.yearly_average("temperature")
ds_temp_yearly["time"] = ds_temp_yearly["time.year"]  # avoids plotting issues

In [3]:
# Get and run epidemiological model
epi_model = epimod.get_example_model(epi_model_name)
ds_months_suitable = ds_clim.climepi.run_epi_model(
    epi_model, return_months_suitable=True
)
ds_months_suitable["time"] = ds_months_suitable["time.year"]  # avoids plotting issues

In [4]:
# Example scenarios and models
scenario_ex = "ssp370"
model_ex = "ipsl-cm6a-lr"
scenarios_compare = ["ssp126", "ssp370", "ssp585"]
scenarios_compare_plot_names = ["SSP1-2.6", "SSP3-7.0", "SSP5-8.5"]
models_compare = ["miroc6", "ipsl-cm6a-lr", "canesm5"]
models_compare_plot_names = ["MIROC6", "IPSL-CM6A-LR", "CanESM5"]

In [5]:
# Internal variability
ds_plot_internal = ds_temp_yearly.sel(scenario=scenario_ex, model=model_ex)
p_internal_plume = ds_plot_internal.climepi.plot_ci_plume(
    polyfit_degree=polyfit_degree,
    kwargs_baseline={"label": "Ensemble mean"},
)
p_internal_ex = ds_plot_internal.climepi.plot_time_series(label="Ensemble member")
p_internal = (p_internal_plume * p_internal_ex).opts(**plot_opts_temp)
p_internal

:Overlay
   .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
   .Curve.Ensemble_mean       :Curve   [time]   (Temperature)
   .Curve.Ensemble_member     :Curve   [time]   (Temperature)

In [6]:
# Model variability
ds_plot_model = (
    ds_temp_yearly.sel(scenario=scenario_ex, model=models_compare)
    .climepi.ensemble_stats(polyfit_degree=polyfit_degree)
    .sel(ensemble_stat="mean", drop=True)
    .assign_coords(model=models_compare_plot_names)
    .rename(model="Climate model")
)
p_model = ds_plot_model.climepi.plot_time_series(by="Climate model").opts(
    **plot_opts_temp
)
p_model

:NdOverlay   [Climate model]
   :Curve   [time]   (Temperature)

In [7]:
# Scenario variability
ds_plot_scenario = (
    ds_temp_yearly.sel(scenario=scenarios_compare)
    .climepi.ensemble_stats(polyfit_degree=polyfit_degree)
    .sel(ensemble_stat="mean", drop=True)
    .mean(dim="model")
    .assign_coords(scenario=scenarios_compare_plot_names)
    .rename(scenario="Scenario")
)
p_scenario = ds_plot_scenario.climepi.plot_time_series(by="Scenario").opts(
    **plot_opts_temp
)
p_scenario

:NdOverlay   [Scenario]
   :Curve   [time]   (temperature)

In [8]:
# Climate variability - plume
p_clim_plume = ds_temp_yearly.climepi.plot_ci_plume(polyfit_degree=polyfit_degree).opts(
    **plot_opts_temp
)
p_clim_plume

:Overlay
   .Area.Scenario_uncertainty :Area   [time]   (scenario_lower,model_lower)
   .Area.I                    :Area   [time]   (model_upper,scenario_upper)
   .Area.Model_uncertainty    :Area   [time]   (model_lower,internal_lower)
   .Area.II                   :Area   [time]   (internal_upper,model_upper)
   .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
   .Curve.Mean                :Curve   [time]   (Temperature)

In [9]:
# Epi variability - plume
p_epi_plume = ds_months_suitable.climepi.plot_ci_plume(
    polyfit_degree=polyfit_degree
).opts(**plot_opts_epi)
p_epi_plume

:Overlay
   .Area.Scenario_uncertainty :Area   [time]   (scenario_lower,model_lower)
   .Area.I                    :Area   [time]   (model_upper,scenario_upper)
   .Area.Model_uncertainty    :Area   [time]   (model_lower,internal_lower)
   .Area.II                   :Area   [time]   (internal_upper,model_upper)
   .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
   .Curve.Mean                :Curve   [time]   (Months where suitability > 0)

In [10]:
make_main_figure([p_internal, p_model, p_scenario, p_clim_plume, p_epi_plume])